In [ ]:
#109.06.24
import pandas as pd
import requests, time
from selenium import webdriver
from pyquery import PyQuery as pq


import openpyxl
from openpyxl import Workbook

# 用python建立一個Excel空白活頁簿
excel_file = Workbook()
# 建立一個工作中表
sheet = excel_file.active

# 先填入第一列的欄位名稱
sheet['A1'] = 'Job'
sheet['B1'] = 'Company'
sheet['C1'] = 'Area'
sheet['D1'] = 'Area2'
sheet['E1'] = 'Jobcat'
sheet['F1'] = 'URL'

In [9]:
###根據預設好的的標籤分類
summaryDict={"1":"人資／經營", 
             "2":"行政╱總務╱法務",
             "3":"行銷╱企劃╱專案管理",
             "4":"金融保險／財會／稽核",
             "5":"客服╱門市╱業務╱貿易",
             "6":"餐飲╱旅遊 ╱美容美髮",
             "7":"資訊軟體系統類",
             "8":"操作╱技術╱維修",
             "9":"資材╱物流╱運輸",
             "10":"營建╱製圖類／測量",
             "11":"傳播╱文字╱設計",
             "12":"醫療／護理／保健",
             "13":"學術╱教育╱輔導",
             "14":"製造／品管／環衛",
             "15":"軍警消╱保全類",
             "16":"其他職務"}
###根據標籤所需要的網址指引
jobcatDict=["2001000000",
            "2002000000",
            "2004000000",
            "2009000000",
"2005000000",
"2006000000",
"2007000000",
"2010000000",
"2011000000",
"2012000000",
"2014000000%2C2013000000",
"2015000000",
"2016000000",
"2009000000",
"2017000000",
"2008000000%2C2018000000"]

###儲存爬蟲data
totalJob=[]
totalCon=[]
totalArea=[]
totalArea2=[]
totalurlList=[]

In [10]:
###要爬蟲所預設的條件
inputData = {'ro':'0', 'keyword':'暑期實習', 'isnew':'30', 'mode':'l','jobcat':"0"}


###在這邊第一個回圈是讓我的inputData改變，來做重複16次的動作（根據要爬的網址不同）
for i in range(1):
    job=[]
    con=[]
    area=[]
    area2=[]
    urlList=[]
    
    inputData['jobcat']= jobcatDict[i]
    url = requests.get("https://www.104.com.tw/jobs/search/?", inputData, allow_redirects=False).url
    driver = webdriver.Chrome(executable_path="/Users/Ponpon/Downloads/chromedriver")
    driver.get(url)
    
    ###因為是動態網頁（固定將頁面捲到頁底）
    for j in range(10):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(0.5)
    
    
    ###移除掉廣告雜訊的data
    pqSource = pq(driver.page_source)
    pqSource.remove(".b-block--ad")
    
    ########工作名稱的解析
    jobvar=0
    for k in pqSource('.job-mode__jobname')('a').items():
        jobvar += 1
        #print("{} {}".format(i,j.text()))
        job.append(k.text())       ###將工作名稱加到job的dataList裡
        totalJob.append(k.text())
        
    ########公司名稱的解析

    
    companyVar=0
    for l in pqSource('.job-mode__company')('a').items():
        companyVar += 1
        #print("{} {}".format(i,j.text()))
        con.append(l.text())       ###將公司名稱加到con的dataList裡
        totalCon.append(l.text()) 
        
     ########地區名稱的解析

    areaVar=0
    for m in pqSource('.job-mode__area').items():
        if areaVar==0:
            areaVar += 1
            continue
        #print("{} {}".format(i,j.text()))
        areaVar += 1
        area.append(m.text()[0:3])      ###將地區名稱加到area的dataList裡
        totalArea.append(m.text()[0:3])
        
    ########行政區的解析
    area2Var=0
    for p in pqSource('.job-mode__area').items():
        if area2Var==0:
            area2Var += 1
            continue
        #print("{} {}".format(i,j.text()))
        area2Var += 1
        area2.append(p.text()[3:])      ###將地區名稱加到area2的dataList裡
        totalArea2.append(p.text()[3:])
    
    ########URL的補充
    for q in pqSource('.job-mode__jobname')("a").items():
        h=q.attr['href']
        #print(h[2:])
        urlList.append( str(h[2:])  )     ###將工作名稱加到job的dataList裡
        totalurlList.append(str(h[2:]))
    
        
    # 使用迴圈逐列增加，使用append的方式往下新增資料到excel
    for n in range(len(job)):
        columnA = job[n]
        columnB = con[n]
        columnC = area[n]
        columnD = area2[n]
        columnE = summaryDict[str(i+1)]
        columnF = urlList[n]
    #     實際將資料寫入每一列
        sheet.append([columnA, columnB, columnC, columnD, columnE, columnF])
    print("已處理完{}/16筆資料".format(i+1))
        
## 儲存成XLSX檔
excel_file.save('Jobdata.xlsx')

print("總共有：{}筆職缺已新增完成".format(len(totalJob)))

已處理完1/16筆資料
總共有：16筆職缺已新增完成


In [11]:
######用來debug檢查工作用

k=1
for i in range(len(job)):
    print("data"+str(k))
    print("Job:{}".format(job[i]))
    print("Company:{}".format(con[i]))
    print("Area:{}".format(area[i]))
    print("")
    k+=1

data1
Job:人資課-暑期實習生
Company:瀚宇博德股份有限公司
Area:桃園市

data2
Job:[力成科技] 2020未來之星暑期實習
Company:力成科技股份有限公司
Area:新竹縣

data3
Job:Want Talent【2020業務工程師學生暑期實習計劃】 Sales Engineer Summer Internship Program
Company:文曄集團_文曄科技股份有限公司
Area:新北市

data4
Job:【2020暑期實習】人資招募助理 (可配合3.5天up/不限科系/應屆可轉正)
Company:新加坡商立可人事顧問有限公司台灣分公司
Area:台北市

data5
Job:人資實習生_約聘職(嚮往人資工作的跳板，非暑期打工)
Company:前思資訊有限公司
Area:台北市

data6
Job:【人資實習生】(人資工作跳板第一站，非暑期打工)
Company:東森新媒體控股股份有限公司
Area:台北市

data7
Job:新創實習生/助理秘書(非暑期，需可配合6個月以上)
Company:台灣夯創業商流資訊有限公司
Area:台北市

data8
Job:新加坡商立可人事實習計畫- 人資+social media/業務組 (HR/sales team )
Company:新加坡商立可人事顧問有限公司台灣分公司
Area:台北市

data9
Job:AI 數據服務事業部丨營運實習生(需實習至少3.5天，半年期)
Company:若水國際股份有限公司
Area:台北市

data10
Job:南山暑期實習計劃-人力資源實習生
Company:(總公司)南山人壽保險股份有限公司
Area:台北市

data11
Job:研發部-暑期實習生
Company:瀚宇博德股份有限公司
Area:桃園市

data12
Job:實習生 Intern
Company:桃園喜來登_桃禧航空城大飯店股份有限公司
Area:桃園市

data13
Job:國外業務/直播行銷助理實習生(限相關科系實習生)/獎助學金每月$5000~$10000
Company:格敦企業股份有限公司
Area:桃園市

data14
Job:實習生 Internship
Company:amba Hotels & Resorts_意舍酒店

In [42]:
#######讀取資料
df = pd.read_excel("Jobdata.xlsx")
#df.head(15)
df[df.Area == "台北市松山區"]

,Job,Company,Area,Jobcat
6,人資實習生_約聘職(嚮往人資工作的跳板，非暑期打工),前思資訊有限公司,台北市松山區,人資／經營
20,大學生駐校代表（暑期實習月薪五萬）,錠嵂保險經紀人股份有限公司_台北營業處(劉祐瑋),台北市松山區,行政╱總務╱法務
23,大學生駐校代表-網路行銷（暑期實習月薪五萬）,錠嵂保險經紀人股份有限公司_台北營業處(劉祐瑋),台北市松山區,行政╱總務╱法務
29,Gogoro Network_Operation Intern 營運實習生,Gogoro Taiwan Limited_睿能創意股份有限公司,台北市松山區,行政╱總務╱法務
30,軟體測試實習生,Gogoro Taiwan Limited_睿能創意股份有限公司,台北市松山區,行政╱總務╱法務
49,(松山) 德國百靈暑期促銷-檔期工讀生 D23,精英人力資源股份有限公司,台北市松山區,行政╱總務╱法務
51,暑期實習生,啟藝文化創意有限公司,台北市松山區,行銷╱企劃╱專案管理
77,【知名能源企業】軟體測試實習生 #大三以上 可轉正,瑞星管理顧問股份有限公司,台北市松山區,金融保險／財會／稽核
78,軟體測試實習生,Gogoro Taiwan Limited_睿能創意股份有限公司,台北市松山區,金融保險／財會／稽核
85,大學生駐校代表（暑期實習月薪五萬）,錠嵂保險經紀人股份有限公司_台北營業處(劉祐瑋),台北市松山區,客服╱門市╱業務╱貿易


In [44]:
a="asdfgh"

In [50]:
a[4:]

'gh'

In [64]:
totalJob

['人資課-暑期實習生',
 '[力成科技] 2020未來之星暑期實習',
 'Want Talent【2020業務工程師學生暑期實習計劃】 Sales Engineer Summer Internship Program',
 '【2020暑期實習】人資招募助理 (可配合3.5天up/不限科系/應屆可轉正)',
 '外商實習-暑期獵才顧問實習生/Recruiter Intern (有機會轉正!)',
 '【人資實習生】(人資工作跳板第一站，非暑期打工)',
 '人資實習生_約聘職(嚮往人資工作的跳板，非暑期打工)',
 '新創實習生/助理秘書(非暑期，需可配合6個月以上)',
 '新加坡商立可人事實習計畫- 人資+social media/業務組 (HR/sales team )',
 'AI 數據服務事業部丨營運實習生(需實習至少3.5天，半年期)',
 '南山暑期實習計劃-人力資源實習生',
 '人資課-暑期實習生',
 '[力成科技] 2020未來之星暑期實習',
 'Want Talent【2020業務工程師學生暑期實習計劃】 Sales Engineer Summer Internship Program',
 '【2020暑期實習】人資招募助理 (可配合3.5天up/不限科系/應屆可轉正)',
 '外商實習-暑期獵才顧問實習生/Recruiter Intern (有機會轉正!)',
 '【人資實習生】(人資工作跳板第一站，非暑期打工)',
 '人資實習生_約聘職(嚮往人資工作的跳板，非暑期打工)',
 '新創實習生/助理秘書(非暑期，需可配合6個月以上)',
 '新加坡商立可人事實習計畫- 人資+social media/業務組 (HR/sales team )',
 'AI 數據服務事業部丨營運實習生(需實習至少3.5天，半年期)',
 '南山暑期實習計劃-人力資源實習生']

In [63]:
job

['人資課-暑期實習生',
 '[力成科技] 2020未來之星暑期實習',
 'Want Talent【2020業務工程師學生暑期實習計劃】 Sales Engineer Summer Internship Program',
 '【2020暑期實習】人資招募助理 (可配合3.5天up/不限科系/應屆可轉正)',
 '外商實習-暑期獵才顧問實習生/Recruiter Intern (有機會轉正!)',
 '【人資實習生】(人資工作跳板第一站，非暑期打工)',
 '人資實習生_約聘職(嚮往人資工作的跳板，非暑期打工)',
 '新創實習生/助理秘書(非暑期，需可配合6個月以上)',
 '新加坡商立可人事實習計畫- 人資+social media/業務組 (HR/sales team )',
 'AI 數據服務事業部丨營運實習生(需實習至少3.5天，半年期)',
 '南山暑期實習計劃-人力資源實習生']

In [17]:
str(h[2:])

'www.104.com.tw/job/6ytz5?jobsource=n104bank2'

In [18]:
a=[]

In [19]:
a.append(str(h[2:]))

In [20]:
a

['www.104.com.tw/job/6ytz5?jobsource=n104bank2']

In [37]:
totalurlList

['www.104.com.tw/job/6ytz5?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6w7jj?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6q5q3?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6na9m?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6z543?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6z1wf?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6tcy6?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6o8li?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6ywev?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6hezw?jobsource=jolist_c_relevance',
 'www.104.com.tw/job/6ytyl?jobsource=joblist_morej',
 'www.104.com.tw/job/5gnb8?jobsource=joblist_morej',
 'www.104.com.tw/job/6qd15?jobsource=joblist_morej',
 'www.104.com.tw/job/6ke7q?jobsource=joblist_morej',
 'www.104.com.tw/job/6qfpp?jobsource=joblist_morej',
 'www.104.com.tw/job/6ye2t?jobsource=joblist_morej']

In [6]:
totalArea

['桃園市',
 '新竹縣',
 '新北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '桃園市',
 '桃園市',
 '桃園市',
 '台北市',
 '台北市',
 '台北市']

In [7]:
area

['桃園市',
 '新竹縣',
 '新北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '台北市',
 '桃園市',
 '桃園市',
 '桃園市',
 '台北市',
 '台北市',
 '台北市']